# Lab 02: Training a Custom Model


**Objective of this lab**: training a small custom model on the Tiny-ImageNet dataset.

## Dataset preparation

In [ ]:
import zipfile, os, wget, zipfile

url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
zip_path = "tiny-imagenet-200.zip"

if not os.path.exists(zip_path):
    
    wget.download(url, zip_path)
    print("\nDownload completed.")
else:
    print("File already exists.")

extract_path = "tiny-imagenet"
if not os.path.exists(os.path.join(extract_path, "tiny-imagenet-200")):
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Estraction completed")
else:
    print("File already exists.")


We need to adjust the format of the val split of the dataset to be used with ImageFolder.

In [ ]:
import os
import shutil

with open('tiny-imagenet/tiny-imagenet-200/val/val_annotations.txt') as f:
    for line in f:
        fn, cls, *_ = line.split('\t')
        os.makedirs(f'tiny-imagenet/tiny-imagenet-200/val/{cls}', exist_ok=True)

        shutil.copyfile(f'tiny-imagenet/tiny-imagenet-200/val/images/{fn}', f'tiny-imagenet/tiny-imagenet-200/val/{cls}/{fn}')

shutil.rmtree('tiny-imagenet/tiny-imagenet-200/val/images')

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((224, 224)),  # Resize to fit the input dimensions of the network
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


tiny_imagenet_dataset_train = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/train', transform=transform)
tiny_imagenet_dataset_val = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/val', transform=transform)

In [ ]:
print(f"Length of train dataset: {len(tiny_imagenet_dataset_train)}")
print(f"Length of val dataset: {len(tiny_imagenet_dataset_val)}")

# The following code also checks the number of samples per class
from collections import Counter

class_counts = Counter([target for _, target in tiny_imagenet_dataset_val])
for class_label, count in class_counts.items():
  print(f"Class {class_label}: {count} entries")


In [ ]:
import torch
train_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_train, batch_size=32, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_val, batch_size=32, shuffle=False)

## Training and validation steps

In [ ]:
from torch import nn
from train import train 
from eval import validate
from models.CustomNet import CustomNet 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = CustomNet().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

best_acc = 0

# Run the training process for num_epochs
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    train(epoch, model, train_loader, criterion, optimizer, device)

    # At the end of each training iteration, perform a validation step
    val_accuracy = validate(model, val_loader, criterion, device)

    # Best validation accuracy
    best_acc = max(best_acc, val_accuracy)


print(f'Best validation accuracy: {best_acc:.2f}%')
